# Modelado de texto

## Preprocesamiento de Texto

### Normalizar Texto

In [ ]:
from nltk.corpus import stopwords as list_stopwords
import unicodedata
import re

PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|@—"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    minusculas=True,
                    punct=True,
                    accents=True,
                    num=True,
                    menciones=True,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """

    if minusculas:
        input_str = input_str.lower()

    if menciones:
        # Expresión regular para encontrar @ seguido de caracteres de palabra (\w+)
        input_str = re.sub(r'@\w+', '', input_str, flags=re.IGNORECASE)
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0

    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue

        if punct:
            if c in SKIP_SYMBOLS:
                continue

        if accents and unicodedata.combining(c):
            continue

        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        
        n_str.append(c)
        c_prev = c

    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    texto = texto.replace('\n', ' ').replace('\r', ' ')
    return texto


from nltk.corpus import stopwords

def eliminar_stopwords(texto, idioma="spanish"):
    _STOPWORDS = stopwords.words(idioma)
    texto = [t for t in texto.split() if t not in _STOPWORDS]
    return " ".join(texto)


### Tokenización
Obtener las oraciones o tokens del texto 

In [ ]:
import nltk

def tokenizar_por_oracion(texto):
    tokenizador_oraciones = nltk.data.load('tokenizers/punkt/spanish.pickle')
    oraciones = tokenizador_oraciones.tokenize(texto)
    return oraciones


from  nltk import word_tokenize
from nltk.stem import SnowballStemmer

def tokenizador_palabra(texto):
    tokens = word_tokenize(texto)
    return tokens

### Stemming - Lematización
reducir las palabras a su raíz

In [ ]:
from nltk.stem import SnowballStemmer

def stemming(texto, idioma):
    stemmer = SnowballStemmer(idioma)
    texto = stemmer.stem(texto)
    return texto


import spacy
from spacy.lang.es.examples import sentences 

def lematizar_texto(input_str):
    NLP_ES = spacy.load("es_core_news_md")
    doc = NLP_ES(input_str)
    lemas = (token.lemma_ for token in doc)
    return " ".join(lemas)


import pandas as pd

def lematizar_dataframe(df, columna_texto, n_hilos=4):
    """
    Aplica la lematización a una columna de un DataFrame usando nlp.pipe() 
    para procesamiento paralelo.

    Args:
        df (pd.DataFrame): DataFrame de entrada.
        columna_texto (str): Nombre de la columna que contiene el texto.
        n_hilos (int): Número de procesos (hilos) a utilizar.

    Returns:
        pd.Series: Serie con el texto lematizado.
    """
    NLP_ES = spacy.load("es_core_news_md")

    # Obtener el generador de documentos procesados
    docs = NLP_ES.pipe(df[columna_texto], n_process=n_hilos)

    # Convertir los documentos procesados a una lista de lemas
    lemas_list = []
    for doc in docs:
        # Unimos los lemas de cada token en un solo string
        lemas_list.append(" ".join([token.lemma_ for token in doc]))

    return pd.Series(lemas_list, index=df.index)



## Vectorización

### TF-IDF - Embeddings

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords

def vectorizar_tf_idf(df,
                      columna_texto="procesado",
                      stopwords = False,
                      idioma="spanish",
                      max_features=5000):
    """
    df: DataFrame con los textos
    columna_texto: columna donde está el texto preprocesado (string)
    idioma: idioma para las stopwords de NLTK
    max_features: número máximo de términos en el vocabulario
    """

    if stopwords: 
        STOPWORDS = nltk_stopwords.words(idioma)
    else:
        STOPWORDS = None

    tfidf_vectorizer = TfidfVectorizer(
        stop_words=STOPWORDS,
        ngram_range=(1, 1),
        max_features=max_features
    )

    # Usamos la columna ya procesada (normalizada, lematizada, sin stopwords)
    textos = df[columna_texto].values

    # Ajustar y transformar
    tfidf_matriz = tfidf_vectorizer.fit_transform(textos)

    # Nombres de características
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Matriz densa en DataFrame (fila = ejemplo, columna = término)
    tfidf_df = pd.DataFrame(
        tfidf_matriz.toarray(),
        columns=feature_names,
        index=df.index
    )

    # Devolvemos también el vectorizador por si lo necesitas después
    return tfidf_df, tfidf_vectorizer


import fasttext

def vectorizar_embeddings(df, ruta_modelo):
    # Cargar el modelo
    ft = fasttext.load_model(ruta_modelo)

    # Crear los vectores densos para cada texto
    df["embeddings"] = df["procesado"].map(lambda lista_tokens: ft.get_sentence_vector(" ".join(lista_tokens)))

# nuevotf = vectorizar_tf_idf(textos, "spanish")
# nuevoem = vectorizar_embeddings(textos, "./datos/bins/MX.bin")

# Entrenamiento

## Definición de la arquitectura

In [ ]:
import torch.nn as nn
import torch.nn.init as init

class MLPDinamico(nn.Module):
    def __init__(self,
                 input_size,
                 num_clases,
                 neuronas_por_capa,
                 activaciones_ocultas,
                 dropout_rates=None,
                 activacion_salida="softmax"):
        super().__init__()

        # Validaciones
        assert len(activaciones_ocultas) == len(neuronas_por_capa), "Todos los arrays deben tener la misma longitud"

        self.input_size = input_size
        self.output_size = num_clases

        # Mapeo de funciones de activación
        self.activacion_diccionario = {
            'relu': nn.ReLU(),
            'sigmoid': nn.Sigmoid(),
            'tanh': nn.Tanh(),
            'leaky_relu': nn.LeakyReLU(),
            'selu': nn.SELU(),
            ## Clasificaión
            'softmax': nn.Softmax(dim=1),
            'log_softmax': nn.LogSoftmax(dim=1)
        }

        # Dropout
        num_capas_ocultas = len(neuronas_por_capa)
        if dropout_rates is None:
            # asignar dropout en 0, significa que no esta apagando neuronas
            dropout_rates = [0.0] * num_capas_ocultas

        assert len(dropout_rates) == num_capas_ocultas, \
            f"Error en los dropout_rates: esperadas {num_capas_ocultas}, recibidas {len(dropout_rates)}"

        # Construcción dinámica de capas
        capas = []
        tam_entrada_actual = input_size

        for i in range(num_capas_ocultas):
            n_salida = neuronas_por_capa[i]
            activacion_str = activaciones_ocultas[i]
            dropout_rate = dropout_rates[i]

            # Capa lineal
            capas.append(nn.Linear(tam_entrada_actual, n_salida))

            # Funció de activación
            capas.append(self._get_activacion(activacion_str))

            # Dropout
            if dropout_rate > 0:
                capas.append(nn.Dropout(dropout_rate))

            # Pasar a la siguiente capa
            tam_entrada_actual = n_salida

        # Ultima capa oculta
        capas.append(nn.Linear(tam_entrada_actual, self.output_size))

        # Activación de salida
        if activacion_salida:
            capas.append(self._get_activacion(activacion_salida))

        self.model = nn.Sequential(*capas)

    def _get_activacion(self, activacion):
        if activacion.lower() in self.activacion_diccionario:
            return self.activacion_diccionario[activacion.lower()]
        else:
            raise ValueError(f"Funcion de activacion no implementada {activacion}")
        
    def inicializar_pesos(self):
        for m in self.modules():
            # **Asegúrate de que 'm' es una capa Lineal antes de acceder a .weight y .bias**
            if isinstance(m, nn.Linear): 
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
        
    def forward(self, X):
        return self.model(X)

## Cargar datos

In [ ]:
import pandas as pd

datos = pd.read_json("./datos/dataset_polaridad_es.json", lines=True)

## Procesamiento de datos

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import os

# Normalizar texto
datos["procesado"] = datos["text"].apply(lambda x: normaliza_texto(x, punct=False, accents=False, minusculas=True))

# Lematizar
datos["procesado"] = lematizar_dataframe(df=datos, columna_texto="procesado", n_hilos=8)

# # Stopwords
# datos["procesado"] = datos["procesado"].apply(eliminar_stopwords)

# Pesado de datos (vectorizar)
# datos["procesado"] = datos["procesado"].apply(tokenizador_palabra)


# ==== Vectorizar ====
#Vectorizar con fastText
vectorizar_embeddings(datos, "./datos/bins/MX.bin")

# Crear matriz de datos
datos["embeddings"].to_numpy()
datos["embeddings"].to_numpy().shape

# Matriz de caracteristicas
X = np.vstack(datos['embeddings'].to_numpy())
Y = datos["klass"].to_numpy()

# Vectorizar con TF-IDF
# print("Vectorizando con TF-IDF...")
# tfidf_df, tfidf_vectorizer = vectorizar_tf_idf(
#     df=datos,
#     stopwords=False,
#     columna_texto="procesado",  # usamos el texto ya normalizado/lematizado/sin stopwords
#     idioma="spanish",
#     max_features=30000
# )

# print("   TF-IDF listo. Forma de la matriz:", tfidf_df.shape)

# # Matriz de características (numpy)
# X = tfidf_df.to_numpy().astype(np.float32)
# Y = datos["klass"].to_numpy()

# Codificar las clases
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

# Separar evaluaciones
X_train, X_val, Y_train_num, Y_val_num = train_test_split(
    X,
    Y_encoded, 
    test_size=0.1, 
    stratify=Y_encoded, 
    random_state=42
)

print(f"División completada. Train: {X_train.shape[0]}, Val: {X_val.shape[0]}")

## Convertir a tensores

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convertir a tensores
X_train_tensor = torch.from_numpy(X_train).float()       # features -> float32
X_val_tensor   = torch.from_numpy(X_val).float()

Y_train_tensor = torch.from_numpy(Y_train_num).long()    # etiquetas -> long (para CrossEntropyLoss)
Y_val_tensor   = torch.from_numpy(Y_val_num).long()


# Crear datasets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, Y_val_tensor)

# Crear DataLoaders
batch_size = 64  # puedes ajustar esto

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


INPUT_SIZE  = X_train.shape[1]          # dimensión del embedding - TF-IDF
NUM_CLASSES = len(np.unique(Y_encoded)) # o len(le.classes_)


## Guardar a JSON

In [ ]:
import json

# --- Función de Ayuda para JSON ---
def append_to_json_file(filename, data):
    """Carga el contenido de un archivo JSON y añade un nuevo registro."""
    try:
        # Cargar los datos existentes
        with open(filename, 'r') as f:
            current_data = json.load(f)
    except FileNotFoundError:
        # Crear la lista si el archivo no existe
        current_data = []
    except json.JSONDecodeError:
        # Manejar archivo vacío o corrupto
        current_data = []

    # Añadir el nuevo dato
    current_data.append(data)

    # Escribir de vuelta al archivo
    with open(filename, 'w') as f:
        json.dump(current_data, f, indent=4)


## Entrenamiento

In [ ]:
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def train_one_model(model,
                    train_loader,
                    val_loader,
                    criterion,
                    optimizer_class,
                    optimizer_kwargs,
                    num_epochs,
                    device,
                    ruido=0.0,
                    early_stopping_patience=7,
                    promedio_metricas="weighted"
                    ):
    """
    Entrena un modelo MLP con:
    - ruido gaussiano opcional en la entrada
    - gradient clipping
    - early stopping según F1 de validación
    """

    model.to(device)
    model.inicializar_pesos()

    # Instanciar optimizador a partir de la clase y kwargs
    optimizer = optimizer_class(model.parameters(), **optimizer_kwargs)

    best_val_acc = 0.0
    best_val_f1  = 0.0
    best_state_dict = None
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        # ---------- FASE DE ENTRENAMIENTO ----------
        model.train()
        train_losses = []
        y_true_train = []
        y_pred_train = []

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            if ruido > 0:
                ruido_batch = torch.randn_like(X_batch) * ruido
                X_batch_input = X_batch + ruido_batch
            else:
                X_batch_input = X_batch

            optimizer.zero_grad()

            outputs = model(X_batch_input)     # logits
            loss = criterion(outputs, y_batch)

            loss.backward()

            # Evitar explosión de gradientes
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_losses.append(loss.item())

            # Predicciones
            preds = outputs.argmax(dim=1)

            y_true_train.extend(y_batch.detach().cpu().numpy())
            y_pred_train.extend(preds.detach().cpu().numpy())

        # Métricas de entrenamiento
        train_loss = float(np.mean(train_losses))
        train_acc  = accuracy_score(y_true_train, y_pred_train)
        train_prec = precision_score(y_true_train, y_pred_train,
                                     average=promedio_metricas, zero_division=0)
        train_rec  = recall_score(y_true_train, y_pred_train,
                                  average=promedio_metricas, zero_division=0)
        train_f1   = f1_score(y_true_train, y_pred_train,
                              average=promedio_metricas, zero_division=0)

        # ---------- FASE DE VALIDACIÓN ----------
        model.eval()
        val_losses = []
        y_true_val = []
        y_pred_val = []

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                val_losses.append(loss.item())

                preds = outputs.argmax(dim=1)

                y_true_val.extend(y_batch.detach().cpu().numpy())
                y_pred_val.extend(preds.detach().cpu().numpy())

        val_loss = float(np.mean(val_losses))
        val_acc  = accuracy_score(y_true_val, y_pred_val)
        val_prec = precision_score(y_true_val, y_pred_val,
                                   average=promedio_metricas, zero_division=0)
        val_rec  = recall_score(y_true_val, y_pred_val,
                                average=promedio_metricas, zero_division=0)
        val_f1   = f1_score(y_true_val, y_pred_val,
                            average=promedio_metricas, zero_division=0)

        # ---------- LOG ----------
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f" Train  -> Loss: {train_loss:.4f} | Acc: {train_acc:.4f} "
              f"| Prec: {train_prec:.4f} | Rec: {train_rec:.4f} | F1: {train_f1:.4f}")
        print(f" Val    -> Loss: {val_loss:.4f} | Acc: {val_acc:.4f} "
              f"| Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")

        # ---------- EARLY STOPPING (por F1 de validación) ----------
        improved = False

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            improved = True

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # si quieres que early stopping dependa solo de F1, comenta esta línea

        if improved:
            # Guardamos mejor modelo en CPU para no llenar GPU
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= early_stopping_patience:
            print(f"\n⏹ Early stopping en la época {epoch+1} "
                  f"(paciencia = {early_stopping_patience})")
            break

    # Cargar el mejor modelo encontrado
    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)

    return best_val_acc


## Evaluación

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluar_modelo(model, data_loader, device):
    """
    Calcula Accuracy, Precision, Recall y F1 sobre un DataLoader.
    """
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average="macro", zero_division=0)
    rec = recall_score(all_labels, all_preds, average="macro", zero_division=0)
    f1  = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return acc, prec, rec, f1


## Función de Grid Search

In [ ]:
from itertools import product
from torch.utils.data import DataLoader  # por si acaso

def generar_combinaciones(param_grid):
    keys = list(param_grid.keys())
    values = list(param_grid.values())
    for vals in product(*values):
        yield dict(zip(keys, vals))


RESULTS_JSON_FILE = "./resultados/resultados_Embeddings.json"

def grid_search_mlp(input_size,
                    num_clases,
                    train_loader,
                    val_loader,
                    param_grid,
                    num_epochs=20,
                    device="cpu",
                    optimizer_class=optim.Adam,
                    criterion=None,
                    activacion_salida=None,
                    ruido=0.2):

    if criterion is None:
        criterion = nn.CrossEntropyLoss()

    best_score = 0.0
    best_params = None
    best_state_dict = None

    # Usamos los datasets base para recrear DataLoaders con distinto batch_size
    base_train_dataset = train_loader.dataset
    base_val_dataset   = val_loader.dataset

    for combo in generar_combinaciones(param_grid):
        print("\nProbando combinación de hiperparámetros:")
        for k, v in combo.items():
            print(f"  {k}: {v}")

        # ---- Arquitectura de la MLP ----
        config_capas = combo["config_capas"]
        neuronas_por_capa    = config_capas["neuronas_por_capa"]
        activaciones_ocultas = config_capas["activaciones_ocultas"]
        dropout_rates        = config_capas.get("dropout_rates", None)

        # ---- Hiperparámetros del optimizador ----
        # OJO: el param_grid usa "learning_rate"
        lr           = combo.get("learning_rate", 1e-3)
        weight_decay = combo.get("weight_decay", 0.0)
        beta1        = combo.get("beta1", 0.9)

        # ---- Batch size desde el grid ----
        # Si no está en combo, usamos el batch_size del loader original o 64
        batch_size = combo.get("batch_size",
                               getattr(train_loader, "batch_size", 64))

        # Crear nuevos DataLoaders con este batch_size
        train_loader_combo = DataLoader(base_train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)
        val_loader_combo   = DataLoader(base_val_dataset,
                                        batch_size=batch_size,
                                        shuffle=False)

        # ---- Crear el modelo ----
        model = MLPDinamico(
            input_size=input_size,
            num_clases=num_clases,
            neuronas_por_capa=neuronas_por_capa,
            activaciones_ocultas=activaciones_ocultas,
            dropout_rates=dropout_rates,
            activacion_salida=activacion_salida
        )

        optimizer_kwargs = {
            "lr": lr,
            "weight_decay": weight_decay,
            "betas": (beta1, 0.999)   # usamos beta1 del grid
        }

        # ---- Entrenar el modelo (solo entrena, ignoramos su retorno) ----
        train_one_model(
            model=model,
            train_loader=train_loader_combo,
            val_loader=val_loader_combo,
            criterion=criterion,
            optimizer_class=optimizer_class,
            optimizer_kwargs=optimizer_kwargs,
            num_epochs=num_epochs,
            device=device
        )

        # ---- Evaluar métricas completas en validación ----
        val_acc, val_prec, val_rec, val_f1 = evaluar_modelo(
            model,
            val_loader_combo,
            device
        )

        print(
            f"Métricas validación -> "
            f"Accuracy: {val_acc:.4f}, "
            f"Precision: {val_prec:.4f}, "
            f"Recall: {val_rec:.4f}, "
            f"F1: {val_f1:.4f}"
        )

        # ---- Guardar resultado en JSON ----
        json_record = {
            "params": {
                "config_capas": config_capas,
                "learning_rate": float(lr),
                "weight_decay": float(weight_decay),
                "batch_size": int(batch_size),
                "beta1": float(beta1)
            },
            "val_accuracy": float(val_acc),
            "val_precision": float(val_prec),
            "val_recall": float(val_rec),
            "val_f1": float(val_f1)
        }

        append_to_json_file(RESULTS_JSON_FILE, json_record)

        # ---- Actualizar mejor modelo según accuracy de validación ----
        if val_acc > best_score:
            best_score = val_acc
            best_params = combo
            best_state_dict = model.state_dict()

    print("\n===== RESULTADOS GRID SEARCH =====")
    print(f"Mejor accuracy de validación: {best_score:.4f}")
    print("Mejores parámetros:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")

    return best_params, best_score, best_state_dict

## Ejemplo

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Mover el modelo a la GPU si está disponible
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

param_grid = {
    # Diferentes arquitecturas de la MLP
    "config_capas": [
        {
            "neuronas_por_capa": [128, 64],
            "activaciones_ocultas": ["relu", "relu"],
            "dropout_rates": [0.3, 0.3]
        },
        # Modelo mediano
        {
            "neuronas_por_capa": [256, 128, 64],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.5, 0.5, 0.5]
        },
        # Más neuronas, dropout moderado
        {
            "neuronas_por_capa": [256, 256, 128],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.4, 0.4, 0.4]
        },
        # Modelo grande con bastante regularización
        {
            "neuronas_por_capa": [512, 256, 128],
            "activaciones_ocultas": ["relu", "relu", "relu"],
            "dropout_rates": [0.5, 0.5, 0.5]
        },
    ],

    # Tasas de aprendizaje a probar
    "learning_rate": [1e-4, 3e-4, 1e-3],

    # Regularización L2 (weight decay)
    "weight_decay": [0.0, 1e-4, 1e-3],

    # Batch size
    "batch_size": [128, 64, 32, 16],

    # Opcional: momento de Adam (beta1)
    "beta1": [0.9, 0.95]
}


best_params, best_score, best_state_dict = grid_search_mlp(
    input_size=INPUT_SIZE,
    num_clases=NUM_CLASSES,
    train_loader=train_loader,
    val_loader=val_loader,
    param_grid=param_grid,
    num_epochs=300,
    device=device,
    optimizer_class=optim.Adam,
    criterion=nn.CrossEntropyLoss(),
    activacion_salida=None,   # MUY IMPORTANTE si uso CrossEntropyLoss
    ruido=0.0 # Noise Injection
)

# Generar gráficas

## Cargar y preparar los datos

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# ================================
# 1. Cargar resultados en DataFrames
# ================================

# Ajusta las rutas si tus archivos están en otra carpeta
ruta_tfidf = "resultados/resultados_TF-IDF_Unigramas.json"
ruta_emb  = "resultados/resultados_Embeddings.json"

with open(ruta_tfidf, "r", encoding="utf-8") as f:
    data_tfidf = json.load(f)

with open(ruta_emb, "r", encoding="utf-8") as f:
    data_emb = json.load(f)

df_tfidf = pd.DataFrame(data_tfidf)
df_emb   = pd.DataFrame(data_emb)

# Extraer hiperparámetros dentro de "params"
def expand_params(df):
    params_df = pd.json_normalize(df["params"])
    # Concatenar lado a lado: métricas + params
    df_metrics = df.drop(columns=["params"])
    return pd.concat([df_metrics, params_df], axis=1)

df_tfidf = expand_params(df_tfidf)
df_emb   = expand_params(df_emb)

# Añadir columna que indique la representación
df_tfidf["representacion"] = "TF-IDF"
df_emb["representacion"] = "Embeddings"

# Unir en un solo DataFrame si hace falta
df_all = pd.concat([df_tfidf, df_emb], ignore_index=True)

# Paleta de colores agradable
color_tfidf = "#1f77b4"     # azul
color_emb   = "#ff7f0e"     # naranja
color_extra = "#2ca02c"     # verde

output_dir = "./resultados/graficas"


## Figura 1 — Comparación de F1 entre TF-IDF y Embeddings

In [ ]:

# =========================================================
# FIGURA 1: Comparación de F1 entre TF-IDF y Embeddings
# =========================================================

plt.figure(figsize=(8, 6))

# Datos para cada grupo
f1_tfidf = df_tfidf["val_f1"]
f1_emb   = df_emb["val_f1"]

# Boxplot básico
plt.boxplot(
    [f1_tfidf, f1_emb],
    labels=["TF-IDF", "Embeddings"],
    patch_artist=True
)

# Colorear cajas
for patch, color in zip(plt.gca().artists, [color_tfidf, color_emb]):
    patch.set_facecolor(color)
    patch.set_edgecolor("black")
    patch.set_alpha(0.7)

plt.title("Figura 1. Comparación del F1-score entre TF-IDF y Embeddings")
plt.ylabel("F1-score (validación)")
plt.grid(axis="y", alpha=0.3)

ruta_fig1 = os.path.join(output_dir, "figura1_f1_comparacion.svg")
plt.tight_layout()
plt.savefig(ruta_fig1, format="svg")
plt.close()
print(f"Figura 1 guardada en: {ruta_fig1}")



## Figura 2 — Efecto del learning rate en F1

In [ ]:
# =========================================================
# FIGURA 2: Efecto del Learning Rate en el rendimiento
# =========================================================

plt.figure(figsize=(8, 6))

# Agrupar por learning_rate y representación, promediando F1
lr_tfidf = df_tfidf.groupby("learning_rate")["val_f1"].mean().sort_index()
lr_emb   = df_emb.groupby("learning_rate")["val_f1"].mean().sort_index()

# Plotear líneas
plt.plot(lr_tfidf.index, lr_tfidf.values, marker="o", linestyle="-",
         label="TF-IDF", color=color_tfidf)
plt.plot(lr_emb.index, lr_emb.values, marker="s", linestyle="-",
         label="Embeddings", color=color_emb)

plt.xscale("log")
plt.xlabel("Learning rate (escala log)")
plt.ylabel("F1-score promedio (validación)")
plt.title("Figura 2. Efecto de la tasa de aprendizaje en el F1-score")
plt.grid(alpha=0.3)
plt.legend()

ruta_fig2 = os.path.join(output_dir, "figura2_learning_rate.svg")
plt.tight_layout()
plt.savefig(ruta_fig2, format="svg")
plt.close()
print(f"Figura 2 guardada en: {ruta_fig2}")


## Figura 3 — Influencia del tamaño de batch

In [ ]:
# =========================================================
# FIGURA 3: Influencia del tamaño de batch
# =========================================================

plt.figure(figsize=(8, 6))

# Asegurar orden numérico de batch_size
df_tfidf["batch_size"] = df_tfidf["batch_size"].astype(int)
df_emb["batch_size"]   = df_emb["batch_size"].astype(int)

bs_tfidf = df_tfidf.groupby("batch_size")["val_f1"].mean().sort_index()
bs_emb   = df_emb.groupby("batch_size")["val_f1"].mean().sort_index()

plt.plot(bs_tfidf.index, bs_tfidf.values, marker="o", linestyle="-",
         label="TF-IDF", color=color_tfidf)
plt.plot(bs_emb.index, bs_emb.values, marker="s", linestyle="-",
         label="Embeddings", color=color_emb)

plt.xlabel("Tamaño de batch")
plt.ylabel("F1-score promedio (validación)")
plt.title("Figura 3. Influencia del tamaño de batch en el F1-score")
plt.grid(alpha=0.3)
plt.legend()

ruta_fig3 = os.path.join(output_dir, "figura3_batch_size.svg")
plt.tight_layout()
plt.savefig(ruta_fig3, format="svg")
plt.close()
print(f"Figura 3 guardada en: {ruta_fig3}")


## Figura 4 — Precisión y Recall en las mejores configuraciones

In [ ]:
# =========================================================
# FIGURA 4: Precisión, Recall y F1 para la mejor configuración
# =========================================================

# Mejor fila (máx F1) para cada representación
best_tfidf = df_tfidf.loc[df_tfidf["val_f1"].idxmax()]
best_emb   = df_emb.loc[df_emb["val_f1"].idxmax()]

metrics = ["val_precision", "val_recall", "val_f1"]
labels_metrics = ["Precisión", "Recall", "F1-score"]

val_tfidf = [best_tfidf[m] for m in metrics]
val_emb   = [best_emb[m]   for m in metrics]

x = range(len(metrics))
width = 0.35

plt.figure(figsize=(8, 6))
plt.bar(
    [i - width/2 for i in x],
    val_tfidf,
    width=width,
    label="TF-IDF",
    color=color_tfidf
)
plt.bar(
    [i + width/2 for i in x],
    val_emb,
    width=width,
    label="Embeddings",
    color=color_emb
)

plt.xticks(list(x), labels_metrics)
plt.ylim(0, 1.0)
plt.ylabel("Valor de la métrica")
plt.title("Figura 4. Precisión, Recall y F1 de las mejores configuraciones")
plt.grid(axis="y", alpha=0.3)
plt.legend()

ruta_fig4 = os.path.join(output_dir, "figura4_mejores_configuraciones.svg")
plt.tight_layout()
plt.savefig(ruta_fig4, format="svg")
plt.close()
print(f"Figura 4 guardada en: {ruta_fig4}")

print("✅ Todas las gráficas han sido generadas y guardadas en formato SVG.")
